In [16]:
import pandas as pd
data_incart = pd.read_csv("MIT-BIH Arrhythmia Database.csv",sep=',')
data_choice = pd.DataFrame()
for i in range(500,len(data_incart)):
    if data_choice.shape[1] > 10:
        break
    if data_incart.loc[i].type == 'N':
        data_choice = pd.concat([data_choice,data_incart.loc[i]],axis=1,ignore_index=True)


for i in range(500,len(data_incart)):
    if data_choice.shape[1] > 20:
        break

    if data_incart.loc[i].type == 'VEB':
        data_choice = pd.concat([data_choice,data_incart.loc[i]],axis=1,ignore_index=True)

for i in range(500,len(data_incart)):
    if data_choice.shape[1] > 30:
        break
    if data_incart.loc[i].type == 'SVEB':
        data_choice = pd.concat([data_choice,data_incart.loc[i]],axis=1,ignore_index=True)

for i in range(500,len(data_incart)):
    if data_choice.shape[1] > 40:
        break
    if data_incart.loc[i].type == 'F':
        data_choice = pd.concat([data_choice,data_incart.loc[i]],axis=1,ignore_index=True)


In [9]:
data_incart.shape

(100689, 34)

In [17]:
import pandas as pd
import numpy as np

n_indices = data_incart[data_incart['type'] == 'N'].index

rng = np.random.default_rng(seed=42)

drop_indices = rng.choice(n_indices, size=len(n_indices) // 2, replace=False)

df = data_incart.drop(index=drop_indices).reset_index(drop=True)

In [18]:
data_choice = data_choice.T

In [19]:
data_choice.drop(columns='record',axis=1,inplace=True)

In [20]:
data_choice.iloc[40]

type                     F
0_pre-RR               397
0_post-RR            284.0
0_pPeak           0.093997
0_tPeak           0.014295
0_rPeak          -0.289838
0_sPeak          -0.686746
0_qPeak          -0.289838
0_qrs_interval           5
0_pq_interval            5
0_qt_interval           21
0_st_interval           11
0_qrs_morph0     -0.289838
0_qrs_morph1     -0.420912
0_qrs_morph2     -0.536986
0_qrs_morph3     -0.624866
0_qrs_morph4     -0.675929
1_pre-RR               397
1_post-RR            284.0
1_pPeak           0.475344
1_tPeak          -0.076036
1_rPeak           -0.36091
1_sPeak          -0.690533
1_qPeak           -0.36091
1_qrs_interval           3
1_pq_interval            6
1_qt_interval           17
1_st_interval            8
1_qrs_morph0      -0.36091
1_qrs_morph1      -0.36091
1_qrs_morph2     -0.530005
1_qrs_morph3     -0.530005
1_qrs_morph4     -0.643416
Name: 40, dtype: object

### Local_Host

In [4]:
import json
import requests
import pandas as pd
from collections import Counter
import random

model = 'deepseek-r1:7b'

def generate(prompt, context):
    r = requests.post(
        'http://localhost:11434/api/generate',
        json={
            'model': model,
            'prompt': prompt,
            'context': context,
            "temperature": 1
        },
        stream=True
    )
    r.raise_for_status()

    full_response = ""
    for line in r.iter_lines():
        body = json.loads(line)
        response_part = body.get('response', '')
        full_response += response_part
        print(response_part, end='', flush=True)

        if 'error' in body:
            raise Exception(body['error'])

        if body.get('done', False):
            return body['context'], full_response.strip()

def main():

    predictions = []

    random_indices = random.sample(range(len(data_choice)), min(50, len(data_choice)))

    for i, row_idx in enumerate(random_indices):
        data_choice_without_type = data_choice.drop(['type'],axis=1)
        ecg_values = data_choice_without_type.loc[row_idx].tolist()
        actual_type = data_choice.loc[row_idx, 'type']
        context = []
        # prompt = (
        #     # f"You are working with the ECG Arrhythmia Classification Dataset, which contains ECG (electrocardiogram) recordings used to classify different types of heart rhythms. Each sample consists of a sequence of ECG amplitude values."
        #     f"You working with the ECG Arrhythmia Classification Dataset, which contains features extracted two-lead ECG signal (lead II, V) from the MIT-BIH Arrhythmia dataset (Physionet). In addition, we have programmatically extracted relevant features from ECG signals to classify regular/irregular heartbeats."
        #     f"The dataset includes the following rhythm classes:"
        #     f"N – Normal rhythm, SVEB – Supraventricular ectopic beat, VEB – Ventricular ectopic beat, F – Fusion beat, Q – Unknown beat"
        #     f"The remaining 32 columns contain 17 features for each ECG lead (17 features for lead-II and 17 features for lead-V5)"
        #     # f"In some cases, the input may also include RR intervals — these are the time durations between successive R-peaks in the ECG signal"
        #     f"Given the ECG signal data below, classify it into one of the five rhythm classes listed above. ECG data: {ecg_values} Respond with only the predicted class in the following format: Answer: <class>"
        # )
        prompt = (
            f"You are a highly specialized medical AI assistant, trained in cardiology with a focus on electrocardiogram (ECG) interpretation and arrhythmia classification. "
            f"Your task is to analyze ECG features and determine the most likely heart rhythm category.\n\n"

            f"The ECG Arrhythmia Classification Dataset you are working with is based on recordings from the MIT-BIH Arrhythmia database (via PhysioNet). "
            f"Each sample includes features extracted from two ECG leads: Lead II and Lead V5. For each lead, 17 diagnostic features have been extracted, totaling 34 features per sample.\n\n"

            f"The goal is to classify the heartbeat into one of the following rhythm classes:\n"
            f"- N: Normal rhythm\n"
            f"- SVEB: Supraventricular ectopic beat\n"
            f"- VEB: Ventricular ectopic beat\n"
            f"- F: Fusion beat\n"
            f"Features are organized into the following groups:\n"
            f"1. **RR Intervals**:\n"
            f"   - *Average RR*: Mean time between R peaks over a window\n"
            f"   - *RR*: Time between the current and previous R peaks\n"
            f"   - *Post RR*: Time between the current and next R peaks\n"
            f"2. **Heartbeat Interval Features**:\n"
            f"   - *PQ Interval*: Time between the start of the P wave and start of the QRS complex\n"
            f"   - *QT Interval*: Duration from the start of Q to the end of T wave\n"
            f"   - *ST Interval*: Time from end of QRS complex to end of T wave\n"
            f"   - *QRS Duration*: Duration of the QRS complex\n"
            f"3. **Heart Beats Amplitude Features**:\n"
            f"   - *P, Q, R, S, T peaks*: Amplitude of corresponding waveform peaks\n"
            f"4. **Morphology Features**:\n"
            f"   - *QRS morph features 0-4*: Represent shape and structure of the QRS complex in fine detail\n\n"

            f"Based on the ECG feature values below, determine the most likely rhythm class. "
            f"Return only the predicted class in the format: Answer: <class>\n\n"
            f"ECG data:\n{ecg_values}"
        )


        print(f"\n\n📍 Обрабатываем строку {i+1}/{len(random_indices)} (Исходный индекс: {row_idx}, Реальный тип: {actual_type}):")
        context, result = generate(prompt, context)

        if "Answer:" in result:
            label = result.split("Answer:")[-1].strip()
        else:
            label = "unknown"

        predictions.append({
            "index": row_idx,
            "actual_type": actual_type,
            "prediction": label,
        })

        if i == 49:  # Ограничиваем 50 итерациями
            break

    # Подсчёт классов
    class_counts = Counter([p["prediction"] for p in predictions])
    print("\n\n🔢 Статистика по предсказанным классам:")
    for label, count in class_counts.items():
        print(f"Класс {label}: {count} примеров")

    # Сохраняем в файл
    df_predictions = pd.DataFrame(predictions)
    df_predictions.to_csv("predictions_random2.csv", index=False)
    print("\n💾 Результаты сохранены в predictions_random.csv")

if __name__ == "__main__":
    main()



📍 Обрабатываем строку 1/41 (Исходный индекс: 2, Реальный тип: N):


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### API OPEN ROUTER

In [25]:
import requests
import json
import pandas as pd
from collections import Counter
import random

API_KEY = "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"
MODEL = "deepseek/deepseek-r1:free"

def process_content(content):
    # Очистка от тегов (если необходимо)
    return content.replace('<think>', '').replace('</think>', '')

def chat_stream(prompt):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "stream": True
    }

    with requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers=headers,
        json=data,
        stream=True
    ) as response:
        if response.status_code != 200:
            print("Ошибка API:", response.status_code)
            return ""

        full_response = []

        for chunk in response.iter_lines():
            if chunk:
                chunk_str = chunk.decode('utf-8').replace('data: ', '')
                try:
                    chunk_json = json.loads(chunk_str)
                    if "choices" in chunk_json:
                        content = chunk_json["choices"][0]["delta"].get("content", "")
                        if content:
                            cleaned = process_content(content)
                            print(cleaned, end='', flush=True)
                            full_response.append(cleaned)
                except Exception as e:
                    continue

        print()
        return ''.join(full_response)

def main():
    predictions = []

    # Случайным образом перемешиваем индексы строк
    random_indices = random.sample(range(len(data_choice)), min(80, len(data_choice)))

    for i, row_idx in enumerate(random_indices):
        data_choice_without_type = data_choice.drop(['type'], axis=1)
        ecg_values = data_choice_without_type.loc[row_idx].tolist()
        actual_type = data_choice.loc[row_idx, 'type']

        # Формирование запроса (промпт) для отправки в API
        prompt = (
            f"You are a highly specialized medical AI assistant, trained in cardiology with a focus on electrocardiogram (ECG) interpretation and arrhythmia classification. "
            f"Your task is to analyze ECG features and determine the most likely heart rhythm category.\n\n"

            f"The ECG Arrhythmia Classification Dataset you are working with is based on recordings from the MIT-BIH Arrhythmia database (via PhysioNet). "
            f"Each sample includes features extracted from two ECG leads: Lead II and Lead V5. For each lead, 17 diagnostic features have been extracted, totaling 34 features per sample.\n\n"

            f"The goal is to classify the heartbeat into one of the following rhythm classes:\n"
            f"- N: Normal rhythm\n"
            f"- SVEB: Supraventricular ectopic beat\n"
            f"- VEB: Ventricular ectopic beat\n"
            f"- F: Fusion beat\n"
            f"Features are organized into the following groups:\n"
            f"1. **RR Intervals**:\n"
            f"   - *Average RR*: Mean time between R peaks over a window\n"
            f"   - *RR*: Time between the current and previous R peaks\n"
            f"   - *Post RR*: Time between the current and next R peaks\n"
            f"2. **Heartbeat Interval Features**:\n"
            f"   - *PQ Interval*: Time between the start of the P wave and start of the QRS complex\n"
            f"   - *QT Interval*: Duration from the start of Q to the end of T wave\n"
            f"   - *ST Interval*: Time from end of QRS complex to end of T wave\n"
            f"   - *QRS Duration*: Duration of the QRS complex\n"
            f"3. **Heart Beats Amplitude Features**:\n"
            f"   - *P, Q, R, S, T peaks*: Amplitude of corresponding waveform peaks\n"
            f"4. **Morphology Features**:\n"
            f"   - *QRS morph features 0-4*: Represent shape and structure of the QRS complex in fine detail\n\n"

            f"Based on the ECG feature values below, determine the most likely rhythm class. "
            f"Return only the predicted class in the format: Answer: <class>\n\n"
            f"ECG data:\n{ecg_values}"
        )

        print(f"\n\n📍 Обрабатываем строку {i+1}/{len(random_indices)} (Исходный индекс: {row_idx}, Реальный тип: {actual_type}):")
        result = chat_stream(prompt)

        if "Answer:" in result:
            label = result.split("Answer:")[-1].strip()
        else:
            label = "unknown"

        predictions.append({
            "index": row_idx,
            "actual_type": actual_type,
            "prediction": label,
        })

        if i == 99:  # Ограничиваем итерациями
            break

    class_counts = Counter([p["prediction"] for p in predictions])
    print("\n\n🔢 Статистика по предсказанным классам:")
    for label, count in class_counts.items():
        print(f"Класс {label}: {count} примеров")

    df_predictions = pd.DataFrame(predictions)
    df_predictions.to_csv("predictions_random.csv", index=False)
    print("\n💾 Результаты сохранены в predictions_random.csv")

if __name__ == "__main__":
    main()




📍 Обрабатываем строку 1/41 (Исходный индекс: 19, Реальный тип: VEB):
Answer: VEB


📍 Обрабатываем строку 2/41 (Исходный индекс: 29, Реальный тип: SVEB):
Answer: VEB


📍 Обрабатываем строку 3/41 (Исходный индекс: 37, Реальный тип: F):


KeyboardInterrupt: 

In [6]:
import pandas as pd
pred = pd.read_csv('Deep_seek.csv',sep=',')
from sklearn.metrics import classification_report
print(classification_report(pred['actual_type'],pred['prediction']))

              precision    recall  f1-score   support

           F       0.00      0.00      0.00        13
           N       0.50      0.47      0.48        15
        SVEB       0.24      0.38      0.29        13
         VEB       0.33      0.47      0.39        15

    accuracy                           0.34        56
   macro avg       0.27      0.33      0.29        56
weighted avg       0.28      0.34      0.30        56



/Users/alexkulbyakov/PycharmProjects/pythonProjects/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alexkulbyakov/PycharmProjects/pythonProjects/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alexkulbyakov/PycharmProjects/pythonProjects/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont